# Module 6: Feature Detection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/06_Module.ipynb)

**Week 11-12: Harris Corners, SIFT, ORB, Feature Matching**

## Learning Objectives
- Understand corner and keypoint detection theory
- Implement Harris corner detector
- Apply SIFT, SURF, and ORB feature detectors
- Perform feature matching and descriptors
- Build practical applications (image stitching, object recognition)

---
## ⚠️ IMPORTANT: Run All Cells in Order

**This notebook must be executed sequentially from top to bottom.**

- Click **Runtime → Run all** (or **Cell → Run All**)
- Do NOT skip cells or run them out of order
- Each cell depends on variables from previous cells

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Feature Detection Theory

### What is a Feature?

A **feature** (or **keypoint**) is a distinctive, repeatable point in an image:
- Can be reliably detected in different images
- Invariant to transformations (rotation, scale, illumination)
- Has a distinctive neighborhood (descriptor)

### Why Feature Detection?

1. **Image matching**: Find corresponding points between images
2. **Object recognition**: Identify objects by their features
3. **3D reconstruction**: Recover 3D structure from 2D images
4. **Tracking**: Follow objects across video frames
5. **Image stitching**: Create panoramas
6. **Camera calibration**: Determine camera parameters

### Feature Types

1. **Corners**: Intersection of edges (high curvature)
2. **Blobs**: Regions with distinctive intensity patterns
3. **Edges**: Boundaries between regions
4. **Ridges**: Elongated structures

### Good Features: Desirable Properties

1. **Repeatability**: Detected consistently across different views
2. **Distinctiveness**: Each feature has unique characteristics
3. **Locality**: Features occupy small, well-defined regions
4. **Quantity**: Enough features for robust matching
5. **Accuracy**: Precise localization
6. **Efficiency**: Fast to compute

In [ ]:
# Load test images
import urllib.request
from urllib.request import Request, urlopen

# Image with corners
url1 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Chessboard480.svg/330px-Chessboard480.svg.png'
req1 = Request(url1, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(req1) as response:
    image_data1 = response.read()
with open('chessboard.png', 'wb') as f:
    f.write(image_data1)

img_chess = cv2.imread('chessboard.png')
img_chess_gray = cv2.cvtColor(img_chess, cv2.COLOR_BGR2GRAY)

# Natural image
url2 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg'
req2 = Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(req2) as response:
    image_data2 = response.read()
with open('box.jpg', 'wb') as f:
    f.write(image_data2)

img_box = cv2.imread('box.jpg')
img_box_gray = cv2.cvtColor(img_box, cv2.COLOR_BGR2GRAY)

print(f"Chessboard image: {img_chess.shape}")
print(f"Box image: {img_box.shape}")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(cv2.cvtColor(img_chess, cv2.COLOR_BGR2RGB))
axes[0].set_title('Chessboard (Many Corners)')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_box, cv2.COLOR_BGR2RGB))
axes[1].set_title('Natural Image')
axes[1].axis('off')

plt.tight_layout()
plt.show()


### 1. Harris Corner Detector

The **Harris corner detector** (Harris & Stephens, 1988) finds corners by analyzing local image gradients.

#### Intuition: What is a Corner?

Consider shifting a small window in different directions:
- **Flat region**: No change in any direction
- **Edge**: Change perpendicular to edge, no change along edge
- **Corner**: Large change in all directions!

#### Mathematical Formulation

**Step 1**: Compute image gradients $I_x$ and $I_y$

**Step 2**: Build the **structure tensor** (second moment matrix):

$$
M = \sum_{(x,y) \in W} \begin{bmatrix}
I_x^2 & I_x I_y \\
I_x I_y & I_y^2
\end{bmatrix}
= \begin{bmatrix}
A & C \\
C & B
\end{bmatrix}
$$

Where:
- $W$ is a local window
- Often weighted by Gaussian $G(\sigma)$

**With Gaussian weighting**:
$$
A = G(\sigma) * I_x^2, \quad B = G(\sigma) * I_y^2, \quad C = G(\sigma) * (I_x I_y)
$$

**Step 3**: Compute corner response

**Harris response function**:
$$
R = \det(M) - k \cdot \text{trace}(M)^2
$$

$$
R = AB - C^2 - k(A + B)^2
$$

Where $k$ is a sensitivity parameter (typically $k = 0.04$ to $0.06$)

#### Eigenvalue Interpretation

The structure tensor $M$ has eigenvalues $\lambda_1, \lambda_2$:

$$
\det(M) = \lambda_1 \lambda_2, \quad \text{trace}(M) = \lambda_1 + \lambda_2
$$

**Classification**:
- $\lambda_1 \approx \lambda_2 \approx 0$: Flat region
- $\lambda_1 \gg \lambda_2$ (or vice versa): Edge
- $\lambda_1, \lambda_2$ both large: **Corner**

**Harris response**:
- $R > 0$: Corner
- $R < 0$: Edge
- $R \approx 0$: Flat

#### Algorithm Steps

1. Compute gradients $I_x$, $I_y$ (e.g., using Sobel)
2. Compute products $I_x^2$, $I_y^2$, $I_x I_y$
3. Apply Gaussian filter to get $A$, $B$, $C$
4. Compute response $R = AB - C^2 - k(A+B)^2$
5. Threshold $R$ to find corners
6. Apply non-maximum suppression

In [ ]:
# Harris corner detection
print("=" * 70)
print("HARRIS CORNER DETECTION")
print("=" * 70)

# Parameters
blockSize = 2  # Neighborhood size
ksize = 3      # Sobel kernel size
k = 0.04       # Harris parameter

# Detect corners
harris_response = cv2.cornerHarris(img_chess_gray, blockSize, ksize, k)

# Dilate to mark corners
harris_response = cv2.dilate(harris_response, None)

print(f"\nHarris parameters:")
print(f"  blockSize: {blockSize} (neighborhood size)")
print(f"  ksize: {ksize} (Sobel aperture)")
print(f"  k: {k} (sensitivity parameter)")
print(f"\nResponse range: [{harris_response.min():.2e}, {harris_response.max():.2e}]")

# Threshold for corner detection
threshold = 0.01 * harris_response.max()
corner_img = img_chess.copy()
corner_img[harris_response > threshold] = [0, 0, 255]  # Red

num_corners = np.sum(harris_response > threshold)
print(f"Corners detected: {num_corners}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(img_chess, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(harris_response, cmap='hot')
axes[1].set_title('Harris Response\n(Brighter = Stronger Corner)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(corner_img, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Detected Corners (Red)\n{num_corners} corners')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Harris detects corners where gradients change in multiple directions!")

In [ ]:
# Demonstrate Harris algorithm step-by-step
print("=" * 70)
print("HARRIS ALGORITHM: STEP-BY-STEP BREAKDOWN")
print("=" * 70)

# Use a simpler image for visualization
test_img = img_chess_gray[:200, :200]

# Step 1: Compute gradients
Ix = cv2.Sobel(test_img, cv2.CV_64F, 1, 0, ksize=3)
Iy = cv2.Sobel(test_img, cv2.CV_64F, 0, 1, ksize=3)
print("Step 1: Computed gradients Ix and Iy using Sobel")

# Step 2: Compute products
Ix2 = Ix ** 2
Iy2 = Iy ** 2
Ixy = Ix * Iy
print("Step 2: Computed Ix², Iy², and IxIy")

# Step 3: Apply Gaussian filter
sigma = 2.0
A = cv2.GaussianBlur(Ix2, (5, 5), sigma)
B = cv2.GaussianBlur(Iy2, (5, 5), sigma)
C = cv2.GaussianBlur(Ixy, (5, 5), sigma)
print(f"Step 3: Applied Gaussian smoothing (σ={sigma})")

# Step 4: Compute Harris response
k = 0.04
det_M = A * B - C ** 2
trace_M = A + B
R = det_M - k * (trace_M ** 2)
print(f"Step 4: Computed Harris response R = det(M) - {k}×trace(M)²")
print(f"  det(M) = AB - C²")
print(f"  trace(M) = A + B")

# Visualize all steps
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

axes[0, 0].imshow(test_img, cmap='gray')
axes[0, 0].set_title('0. Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(Ix, cmap='RdBu')
axes[0, 1].set_title('1. Gradient Ix')
axes[0, 1].axis('off')

axes[0, 2].imshow(Iy, cmap='RdBu')
axes[0, 2].set_title('1. Gradient Iy')
axes[0, 2].axis('off')

axes[1, 0].imshow(Ix2, cmap='hot')
axes[1, 0].set_title('2. Ix²')
axes[1, 0].axis('off')

axes[1, 1].imshow(Iy2, cmap='hot')
axes[1, 1].set_title('2. Iy²')
axes[1, 1].axis('off')

axes[1, 2].imshow(np.abs(Ixy), cmap='hot')
axes[1, 2].set_title('2. |IxIy|')
axes[1, 2].axis('off')

axes[2, 0].imshow(A, cmap='hot')
axes[2, 0].set_title('3. A (smoothed Ix²)')
axes[2, 0].axis('off')

axes[2, 1].imshow(B, cmap='hot')
axes[2, 1].set_title('3. B (smoothed Iy²)')
axes[2, 1].axis('off')

axes[2, 2].imshow(R, cmap='hot')
axes[2, 2].set_title('4. Harris Response R\n(Corners = bright spots)')
axes[2, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Harris response is high where both eigenvalues are large!")

### 2. Shi-Tomasi Corner Detector

**Shi-Tomasi** (1994) improved Harris by using a simpler criterion.

#### Difference from Harris

**Harris response**:
$$
R = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2
$$

**Shi-Tomasi response**:
$$
R = \min(\lambda_1, \lambda_2)
$$

#### Advantages

1. **No parameter $k$**: Eliminates tuning
2. **Better feature selection**: Selects best corners directly
3. **Quality measure**: Direct geometric meaning (minimum gradient variation)

#### When to Use

- **Shi-Tomasi**: Feature tracking, optical flow
- **Harris**: General corner detection, more geometric insights

In [ ]:
# Shi-Tomasi corner detection
print("=" * 70)
print("SHI-TOMASI CORNER DETECTION")
print("=" * 70)

# Parameters
maxCorners = 100
qualityLevel = 0.01
minDistance = 10

# Detect corners
corners = cv2.goodFeaturesToTrack(img_box_gray, maxCorners, qualityLevel, minDistance)

print(f"\nShi-Tomasi parameters:")
print(f"  maxCorners: {maxCorners} (maximum number of corners)")
print(f"  qualityLevel: {qualityLevel} (minimum quality)")
print(f"  minDistance: {minDistance} (minimum distance between corners)")
print(f"\nCorners detected: {len(corners)}")

# Draw corners
img_corners = img_box.copy()
if corners is not None:
    corners = np.int32(corners)
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(img_corners, (x, y), 5, (0, 255, 0), -1)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(cv2.cvtColor(img_box, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_corners, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Shi-Tomasi Corners (Green)\n{len(corners)} corners')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Shi-Tomasi directly selects best corners without parameter k!")

### 3. SIFT (Scale-Invariant Feature Transform)

**SIFT** (Lowe, 2004) is a landmark algorithm for robust feature detection and description.

#### Key Innovations

1. **Scale invariance**: Detects features at multiple scales
2. **Rotation invariance**: Assigns canonical orientation
3. **Distinctive descriptors**: 128-dimensional feature vectors
4. **Robustness**: Invariant to illumination, viewpoint changes

#### Algorithm Overview

**Step 1: Scale-Space Extrema Detection**

Build **Difference of Gaussians (DoG)** pyramid:
$$
D(x, y, \sigma) = [G(x, y, k\sigma) - G(x, y, \sigma)] * I(x, y)
$$

Find local extrema in both **space** and **scale**.

**Step 2: Keypoint Localization**

- Reject low-contrast points
- Eliminate edge responses
- Sub-pixel localization using Taylor expansion

**Step 3: Orientation Assignment**

Compute gradient histogram in neighborhood:
$$
\theta(x, y) = \arctan\left(\frac{L_y}{L_x}\right)
$$

Assign dominant orientation(s) to achieve rotation invariance.

**Step 4: Keypoint Descriptor**

- Rotate neighborhood to canonical orientation
- Divide into 4×4 subregions
- Compute 8-bin gradient histogram per subregion
- Result: **128-dimensional descriptor** (4×4×8)
- Normalize for illumination invariance

#### SIFT Descriptor Properties

- **Distinctive**: High discrimination power
- **Invariant**: Scale, rotation, illumination
- **Robust**: Partial invariance to viewpoint, affine transformations
- **Efficient**: Fast matching using nearest neighbor search

In [ ]:
# SIFT feature detection
print("=" * 70)
print("SIFT FEATURE DETECTION")
print("=" * 70)

# Create SIFT detector
# Note: SIFT is patented and not available in opencv-python
# You need opencv-contrib-python: pip install opencv-contrib-python
try:
    sift = cv2.SIFT_create()
    
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(img_box_gray, None)
    
    print(f"\nKeypoints detected: {len(keypoints)}")
    print(f"Descriptor shape: {descriptors.shape}")
    print(f"  Each keypoint has a {descriptors.shape[1]}-dimensional descriptor")
    
    # Analyze keypoint properties
    scales = [kp.size for kp in keypoints]
    angles = [kp.angle for kp in keypoints]
    responses = [kp.response for kp in keypoints]
    
    print(f"\nKeypoint statistics:")
    print(f"  Scale range: [{min(scales):.2f}, {max(scales):.2f}]")
    print(f"  Angle range: [{min(angles):.2f}°, {max(angles):.2f}°]")
    print(f"  Response range: [{min(responses):.4f}, {max(responses):.4f}]")
    
    # Draw keypoints with different visualization options
    img_keypoints = cv2.drawKeypoints(img_box, keypoints, None,
                                      flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    axes[0].imshow(cv2.cvtColor(img_box, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_keypoints, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'SIFT Keypoints\n{len(keypoints)} features\n(Circle = scale, Arrow = orientation)')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Visualize descriptor statistics
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].hist(scales, bins=30, edgecolor='black')
    axes[0].set_title('Keypoint Scale Distribution')
    axes[0].set_xlabel('Scale (size)')
    axes[0].set_ylabel('Count')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].hist(angles, bins=36, edgecolor='black')
    axes[1].set_title('Keypoint Orientation Distribution')
    axes[1].set_xlabel('Angle (degrees)')
    axes[1].set_ylabel('Count')
    axes[1].grid(True, alpha=0.3)
    
    # Descriptor heatmap (first 20 keypoints)
    axes[2].imshow(descriptors[:20, :], cmap='hot', aspect='auto')
    axes[2].set_title('SIFT Descriptors\n(First 20 keypoints)')
    axes[2].set_xlabel('Descriptor dimension (0-127)')
    axes[2].set_ylabel('Keypoint index')
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey Insight: SIFT detects features at multiple scales with orientation!")
    
except AttributeError:
    print("\nSIFT not available. Install opencv-contrib-python:")
    print("  pip install opencv-contrib-python")
    print("\nNote: SIFT is patented (until 2020) and requires opencv-contrib")

### 4. ORB (Oriented FAST and Rotated BRIEF)

**ORB** (Rublee et al., 2011) is a fast, free alternative to SIFT.

#### Components

**FAST (Features from Accelerated Segment Test)**:
- Fast corner detector
- Compares pixel with circle of 16 neighbors
- Corner if N contiguous pixels are brighter/darker

**BRIEF (Binary Robust Independent Elementary Features)**:
- Binary descriptor (string of 0s and 1s)
- Based on simple intensity comparisons
- Fast to compute and match (Hamming distance)

#### ORB Improvements

1. **Orientation**: Adds rotation invariance to FAST
   - Compute intensity centroid
   - Orientation: $\theta = \arctan(m_{01}/m_{10})$

2. **Steered BRIEF**: Rotates BRIEF pattern according to orientation

3. **Scale pyramid**: Multi-scale detection

#### ORB vs SIFT

| Property | SIFT | ORB |
|----------|------|-----|
| Speed | Slow | **Very Fast** (10-100× faster) |
| Descriptor | 128D float | **256-bit binary** |
| Matching | L2 distance | **Hamming distance** (faster) |
| Patent | Patented | **Free** |
| Rotation invariance | Yes | Yes |
| Scale invariance | Yes | Yes (pyramid) |
| Robustness | **Better** | Good |

**Use ORB when**: Speed is critical, real-time applications

**Use SIFT when**: Maximum robustness needed

In [ ]:
# ORB feature detection
print("=" * 70)
print("ORB FEATURE DETECTION")
print("=" * 70)

# Create ORB detector
orb = cv2.ORB_create(nfeatures=500)

# Detect keypoints and compute descriptors
keypoints_orb, descriptors_orb = orb.detectAndCompute(img_box_gray, None)

print(f"\nKeypoints detected: {len(keypoints_orb)}")
print(f"Descriptor shape: {descriptors_orb.shape}")
print(f"  Each keypoint has a {descriptors_orb.shape[1]}-bit binary descriptor")
print(f"  Descriptor dtype: {descriptors_orb.dtype} (binary)")

# Draw keypoints
img_keypoints_orb = cv2.drawKeypoints(img_box, keypoints_orb, None,
                                      color=(0, 255, 0),
                                      flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(cv2.cvtColor(img_box, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_keypoints_orb, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'ORB Keypoints\n{len(keypoints_orb)} features\n(FAST corners + BRIEF descriptors)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Visualize binary descriptors
fig, ax = plt.subplots(figsize=(12, 6))
ax.imshow(descriptors_orb[:50, :], cmap='binary', aspect='auto')
ax.set_title('ORB Binary Descriptors\n(First 50 keypoints, black=0, white=1)')
ax.set_xlabel('Bit index (0-255)')
ax.set_ylabel('Keypoint index')
plt.tight_layout()
plt.show()

print("\nKey Insight: ORB uses binary descriptors for FAST matching!")

### 5. Feature Matching

**Feature matching** finds corresponding keypoints between images.

#### Matching Strategies

**1. Brute-Force Matching**

Compare every descriptor in image 1 with every descriptor in image 2:
$$
\text{match}(i) = \arg\min_j d(\text{desc}_1[i], \text{desc}_2[j])
$$

Distance metrics:
- **L2 (Euclidean)**: For SIFT, SURF (float descriptors)
- **Hamming**: For ORB, BRIEF, BRISK (binary descriptors)

**2. FLANN (Fast Library for Approximate Nearest Neighbors)**

- Much faster for large descriptor sets
- Approximate but highly accurate
- Uses k-d trees or LSH (Locality-Sensitive Hashing)

#### Ratio Test (Lowe's Test)

Find two nearest neighbors for each descriptor:
$$
\text{ratio} = \frac{d_1}{d_2}
$$

Keep match if $\text{ratio} < 0.7$ (or 0.75)

**Rationale**: Good matches have much closer nearest neighbor than second-nearest

#### Cross-Check

Match is valid only if:
- Descriptor A matches descriptor B
- AND descriptor B matches descriptor A

Reduces false matches significantly.

In [ ]:
# Feature matching demonstration
print("=" * 70)
print("FEATURE MATCHING: ORB + BRUTE FORCE")
print("=" * 70)

# Load two images of the same scene (simulated by rotating)
img1 = img_box.copy()
h, w = img1.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, 30, 0.9)
img2 = cv2.warpAffine(img1, M, (w, h))

img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Detect ORB features
orb = cv2.ORB_create(nfeatures=1000)
kp1, des1 = orb.detectAndCompute(img1_gray, None)
kp2, des2 = orb.detectAndCompute(img2_gray, None)

print(f"\nKeypoints in image 1: {len(kp1)}")
print(f"Keypoints in image 2: {len(kp2)}")

# Brute-force matching with Hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

# Sort by distance (best matches first)
matches = sorted(matches, key=lambda x: x.distance)

print(f"\nTotal matches found: {len(matches)}")
print(f"Best match distance: {matches[0].distance}")
print(f"Worst match distance: {matches[-1].distance}")

# Draw top 50 matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None,
                               flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
plt.title(f'ORB Feature Matching (Top 50 of {len(matches)} matches)\nImage 2 is rotated 30° and scaled 0.9×')
plt.axis('off')
plt.tight_layout()
plt.show()

print("\nKey Insight: Features can be matched even after rotation and scaling!")

In [ ]:
# Demonstrate ratio test (Lowe's test)
print("=" * 70)
print("LOWE'S RATIO TEST FOR ROBUST MATCHING")
print("=" * 70)

# Brute-force matcher without cross-check (to use knnMatch)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

# Find 2 nearest neighbors for each descriptor
matches_knn = bf.knnMatch(des1, des2, k=2)

print(f"\nTotal keypoint pairs: {len(matches_knn)}")

# Apply ratio test
good_matches = []
ratio_threshold = 0.75

for match_pair in matches_knn:
    if len(match_pair) == 2:  # Need 2 neighbors
        m, n = match_pair
        # Lowe's ratio test
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)

print(f"Matches after ratio test (threshold={ratio_threshold}): {len(good_matches)}")
print(f"Rejection rate: {100 * (1 - len(good_matches) / len(matches_knn)):.1f}%")

# Draw good matches
img_good_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches[:50], None,
                                   flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_good_matches, cv2.COLOR_BGR2RGB))
plt.title(f'Matches After Ratio Test (Top 50 of {len(good_matches)})\nRatio threshold = {ratio_threshold}')
plt.axis('off')
plt.tight_layout()
plt.show()

# Visualize ratio distribution
ratios = []
for match_pair in matches_knn:
    if len(match_pair) == 2:
        m, n = match_pair
        if n.distance > 0:
            ratios.append(m.distance / n.distance)

plt.figure(figsize=(10, 5))
plt.hist(ratios, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(ratio_threshold, color='red', linestyle='--', linewidth=2,
            label=f'Threshold = {ratio_threshold}')
plt.xlabel('Ratio (d1 / d2)')
plt.ylabel('Count')
plt.title('Distribution of Distance Ratios\n(Good matches have low ratios)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey Insight: Ratio test filters out ambiguous matches!")

## Summary

### Key Concepts

1. **Corner Detection**: Harris, Shi-Tomasi
2. **Feature Descriptors**: SIFT (128D float), ORB (256-bit binary)
3. **Feature Matching**: Brute-force, FLANN, ratio test
4. **Invariance**: Scale, rotation, illumination

### Detector Comparison

| Method | Type | Invariance | Speed | Descriptor | Best For |
|--------|------|------------|-------|------------|----------|
| **Harris** | Corner | None | Fast | None | Corner detection |
| **Shi-Tomasi** | Corner | None | Fast | None | Tracking, optical flow |
| **SIFT** | Blob | Scale, Rotation | Slow | 128D float | Robustness critical |
| **SURF** | Blob | Scale, Rotation | Medium | 64/128D float | Faster than SIFT |
| **ORB** | Corner | Scale, Rotation | **Very Fast** | 256-bit binary | Real-time, mobile |

### Mathematical Formulas Reference

**Harris corner response**:
$$
R = \det(M) - k \cdot \text{trace}(M)^2 = \lambda_1\lambda_2 - k(\lambda_1 + \lambda_2)^2
$$

**Shi-Tomasi response**:
$$
R = \min(\lambda_1, \lambda_2)
$$

**Structure tensor**:
$$
M = \sum_{W} \begin{bmatrix} I_x^2 & I_xI_y \\ I_xI_y & I_y^2 \end{bmatrix}
$$

**Lowe's ratio test**:
$$
\frac{d_1}{d_2} < 0.75 \quad \text{(keep match)}
$$

### OpenCV Functions Reference

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Harris | `cv2.cornerHarris(img, blockSize, ksize, k)` | k ≈ 0.04-0.06 |
| Shi-Tomasi | `cv2.goodFeaturesToTrack(img, maxCorners, quality, minDist)` | quality ≈ 0.01 |
| SIFT | `cv2.SIFT_create()` | Requires opencv-contrib |
| ORB | `cv2.ORB_create(nfeatures)` | Free, fast |
| BF Matcher | `cv2.BFMatcher(normType, crossCheck)` | NORM_L2 or NORM_HAMMING |
| FLANN | `cv2.FlannBasedMatcher(indexParams, searchParams)` | Fast approximate |

**Next**: Module 7 - Image Segmentation